
# **Ejercicio Semana 6: Características en Imágenes**
Visión por Computador

Ingeniería Agronómica

Monica Paola Vargas Tirado

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

%cd /content/drive/MyDrive/UNIVERSIDAD EAFIT/VISION POR COMPUTADOR/EJERCICIOS/Dataset/

# Cargar las imágenes
img1 = cv2.imread('Imagen_1.jpeg')
img2 = cv2.imread('Imagen_2.jpeg')

def create_panoramica(img1, img2):
    # Inicializar el detector ORB
    orb = cv2.ORB_create()

    # Encontrar los puntos clave y sus descriptores para ambas imágenes
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    # Inicializar el objeto de coincidencia de fuerza bruta
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Realizar la coincidencia de los descriptores
    matches = bf.match(descriptors1, descriptors2)

    # Ordenar las coincidencias por distancia
    matches = sorted(matches, key=lambda x: x.distance)

    # Elegir solo las mejores coincidencias
    good_matches = matches[:50]  # Puedes ajustar este número según tus necesidades

    # Dibujar los emparejamientos en una nueva imagen
    matching_result = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Convertir los puntos clave en formato numpy array
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Calcular la matriz de homografía
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    # Deformar la segunda imagen para alinearla con la primera
    img2_aligned = cv2.warpPerspective(img2, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))

    # Superponer las imágenes
    img2_aligned[0:img1.shape[0], 0:img1.shape[1]] = img1

    return img2_aligned, matching_result

# Crear la foto panorámica y obtener los emparejamientos dibujados
panoramica, matching_result = create_panoramica(img1, img2)

# Mostrar la foto panorámica y los emparejamientos
cv2_imshow(matching_result)

In [ ]:
cv2_imshow(panoramica)